# Autoencoder for Snippets
We build a mirrored deep convultional autoencoder.

In [2]:
from Variational_Autoencoder_alla_Valerio import VAE

## Load the Data
First we need to load the snippets we created.

In [ ]:
subfolder = "0.25_16"
x_train = np.load("data_and_models\\" + subfolder + "\\spectos.npy")
#x_train = x_train[:1000]


## Building the VAE

In [ ]:
autoencoder = VAE(
    input_shape=(x_train[0].shape[0], x_train[0].shape[1], x_train[0].shape[2]),
    conv_filters=(512, 256, 128, 64, 32),
    conv_kernels=(3, 3, 3, 3, 3),
    conv_strides=(2, 2, 2, 2, (2,1)),
    latent_space_dim=128
)

autoencoder.summary()

print(x_train.shape)



## Train the VAE
We recommend to train the Autoencoder in Pycharm as Jupyter might lose the connection during training.

In [ ]:
# LEARNING_RATE = 0.0005
# BATCH_SIZE = 32
# EPOCHS = 20

# autoencoder.compile_model(LEARNING_RATE)
# autoencoder.train(x_train, BATCH_SIZE, EPOCHS)

## Save the trained VAE

In [ ]:
# save_path = "data_and_models\\" + subfolder + "\\VAE_Vocals_" + str(autoencoder.latent_space_dim) + "D_" + str(autoencoder.num_of_train_data) + "samples_" + str(EPOCHS) + "Epochs"
# autoencoder.save(save_path)